In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Transfer Learning Comparison\n",
    "## Этап 3: Сравнение разных подходов transfer learning\n",
    "\n",
    "В этом ноутбуке мы:\n",
    "1. Сравним минимум 2 разные pretrained модели (ResNet18, VGG16)\n",
    "2. Сравним feature extraction vs fine-tuning\n",
    "3. Проведем эксперименты с разными learning rates\n",
    "4. Протестируем оптимизаторы с different LR для разных частей модели"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import torch\n",
    "import torch.nn as nn\n",
    "import torch.optim as optim\n",
    "from torch.utils.data import DataLoader\n",
    "from torchvision import datasets, transforms\n",
    "import matplotlib.pyplot as plt\n",
    "import numpy as np\n",
    "import pandas as pd\n",
    "import time\n",
    "import sys\n",
    "import os\n",
    "\n",
    "# Добавляем путь к проекту\n",
    "sys.path.append('..')\n",
    "\n",
    "from models.transfer_models import get_model\n",
    "from utils.optimizers import get_optimizer_with_different_lr, get_optimizer_simple\n",
    "from training.train_transfer import train_model, plot_training_history\n",
    "\n",
    "device = torch.device(\"cuda\" if torch.cuda.is_available() else \"cpu\")\n",
    "print(f'Using device: {device}')"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Подготовка данных"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Параметры\n",
    "DATA_DIR = '../data/animal_faces'\n",
    "BATCH_SIZE = 32\n",
    "NUM_EPOCHS = 15\n",
    "IMAGE_SIZE = 224\n",
    "NUM_CLASSES = 3\n",
    "\n",
    "# Трансформации\n",
    "data_transforms = {\n",
    "    'train': transforms.Compose([\n",
    "        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),\n",
    "        transforms.RandomHorizontalFlip(),\n",
    "        transforms.RandomRotation(10),\n",
    "        transforms.ToTensor(),\n",
    "        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])\n",
    "    ]),\n",
    "    'val': transforms.Compose([\n",
    "        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),\n",
    "        transforms.ToTensor(),\n",
    "        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])\n",
    "    ])\n",
    "}\n",
    "\n",
    "# Загрузка данных\n",
    "image_datasets = {\n",
    "    x: datasets.ImageFolder(os.path.join(DATA_DIR, x), data_transforms[x])\n",
    "    for x in ['train', 'val']\n",
    "}\n",
    "\n",
    "dataloaders = {\n",
    "    x: DataLoader(image_datasets[x], batch_size=BATCH_SIZE, \n",
    "                 shuffle=(x == 'train'), num_workers=4)\n",
    "    for x in ['train', 'val']\n",
    "}\n",
    "\n",
    "print(f\"Train size: {len(image_datasets['train'])}\")\n",
    "print(f\"Val size: {len(image_datasets['val'])}\")\n",
    "print(f\"Classes: {image_datasets['train'].classes}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Эксперимент 1: Сравнение разных архитектур\n",
    "### Сравниваем ResNet18 и VGG16 в режиме feature extraction"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Результаты для сравнения\n",
    "results_architectures = {}\n",
    "\n",
    "# Список моделей для тестирования\n",
    "models_to_test = ['resnet18', 'vgg16']\n",
    "\n",
    "for model_name in models_to_test:\n",
    "    print(f\"\\n{'='*60}\")\n",
    "    print(f\"Training {model_name} with feature extraction\")\n",
    "    print(f\"{'='*60}\")\n",
    "    \n",
    "    # Создаем модель\n",
    "    model = get_model(model_name, num_classes=NUM_CLASSES, \n",
    "                     pretrained=True, mode='feature_extraction')\n",
    "    \n",
    "    # Оптимизатор и loss\n",
    "    criterion = nn.CrossEntropyLoss()\n",
    "    optimizer = optim.Adam(model.get_trainable_params(), lr=0.001)\n",
    "    \n",
    "    # Обучение\n",
    "    save_path = f'../checkpoints/transfer_{model_name}_feature_extraction.pth'\n",
    "    start_time = time.time()\n",
    "    \n",
    "    trained_model, history = train_model(\n",
    "        model, dataloaders, criterion, optimizer,\n",
    "        num_epochs=NUM_EPOCHS, device=device, \n",
    "        patience=5, save_path=save_path\n",
    "    )\n",
    "    \n",
    "    training_time = time.time() - start_time\n",
    "    \n",
    "    # Сохраняем результаты\n",
    "    results_architectures[model_name] = {\n",
    "        'history': history,\n",
    "        'best_val_acc': max(history['val_acc']),\n",
    "        'training_time': training_time,\n",
    "        'mode': 'feature_extraction'\n",
    "    }\n",
    "    \n",
    "    print(f\"\\nBest validation accuracy: {results_architectures[model_name]['best_val_acc']:.4f}\")\n",
    "    print(f\"Training time: {training_time/60:.2f} minutes\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Визуализация сравнения архитектур\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "for model_name in models_to_test:\n",
    "    history = results_architectures[model_name]['history']\n",
    "    \n",
    "    # Loss\n",
    "    axes[0].plot(history['val_loss'], label=f'{model_name}', marker='o', markersize=4)\n",
    "    \n",
    "    # Accuracy\n",
    "    axes[1].plot(history['val_acc'], label=f'{model_name}', marker='o', markersize=4)\n",
    "\n",
    "axes[0].set_xlabel('Epoch')\n",
    "axes[0].set_ylabel('Validation Loss')\n",
    "axes[0].set_title('Model Comparison - Validation Loss')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True)\n",
    "\n",
    "axes[1].set_xlabel('Epoch')\n",
    "axes[1].set_ylabel('Validation Accuracy')\n",
    "axes[1].set_title('Model Comparison - Validation Accuracy')\n",
    "axes[1].legend()\n",
    "axes[1].grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../results/transfer/models_comparison.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(\"Saved to ../results/transfer/models_comparison.png\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Эксперимент 2: Feature Extraction vs Fine-Tuning\n",
    "### Берем лучшую модель и сравниваем два подхода"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Выбираем лучшую модель из предыдущего эксперимента\n",
    "best_model_name = max(results_architectures, \n",
    "                     key=lambda k: results_architectures[k]['best_val_acc'])\n",
    "print(f\"Best model from architecture comparison: {best_model_name}\")\n",
    "\n",
    "# Результаты для сравнения режимов\n",
    "results_modes = {}\n",
    "\n",
    "modes = ['feature_extraction', 'fine_tuning']\n",
    "\n",
    "for mode in modes:\n",
    "    print(f\"\\n{'='*60}\")\n",
    "    print(f\"Training {best_model_name} in {mode} mode\")\n",
    "    print(f\"{'='*60}\")\n",
    "    \n",
    "    # Создаем модель\n",
    "    model = get_model(best_model_name, num_classes=NUM_CLASSES, \n",
    "                     pretrained=True, mode=mode)\n",
    "    \n",
    "    # Оптимизатор (для fine-tuning используем меньший LR)\n",
    "    criterion = nn.CrossEntropyLoss()\n",
    "    \n",
    "    if mode == 'feature_extraction':\n",
    "        optimizer = optim.Adam(model.get_trainable_params(), lr=0.001)\n",
    "    else:  # fine_tuning\n",
    "        optimizer = optim.Adam(model.get_trainable_params(), lr=0.0001)\n",
    "    \n",
    "    # Обучение\n",
    "    save_path = f'../checkpoints/transfer_{best_model_name}_{mode}.pth'\n",
    "    start_time = time.time()\n",
    "    \n",
    "    trained_model, history = train_model(\n",
    "        model, dataloaders, criterion, optimizer,\n",
    "        num_epochs=NUM_EPOCHS, device=device, \n",
    "        patience=5, save_path=save_path\n",
    "    )\n",
    "    \n",
    "    training_time = time.time() - start_time\n",
    "    \n",
    "    # Сохраняем результаты\n",
    "    results_modes[mode] = {\n",
    "        'history': history,\n",
    "        'best_val_acc': max(history['val_acc']),\n",
    "        'training_time': training_time\n",
    "    }\n",
    "    \n",
    "    print(f\"\\nBest validation accuracy: {results_modes[mode]['best_val_acc']:.4f}\")\n",
    "    print(f\"Training time: {training_time/60:.2f} minutes\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Визуализация Feature Extraction vs Fine-Tuning\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "for mode in modes:\n",
    "    history = results_modes[mode]['history']\n",
    "    label = 'Feature Extraction' if mode == 'feature_extraction' else 'Fine-Tuning'\n",
    "    \n",
    "    # Loss\n",
    "    axes[0].plot(history['val_loss'], label=label, marker='o', markersize=4)\n",
    "    \n",
    "    # Accuracy\n",
    "    axes[1].plot(history['val_acc'], label=label, marker='o', markersize=4)\n",
    "\n",
    "axes[0].set_xlabel('Epoch')\n",
    "axes[0].set_ylabel('Validation Loss')\n",
    "axes[0].set_title(f'{best_model_name}: Feature Extraction vs Fine-Tuning - Loss')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True)\n",
    "\n",
    "axes[1].set_xlabel('Epoch')\n",
    "axes[1].set_ylabel('Validation Accuracy')\n",
    "axes[1].set_title(f'{best_model_name}: Feature Extraction vs Fine-Tuning - Accuracy')\n",
    "axes[1].legend()\n",
    "axes[1].grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../results/transfer/feature_extraction_vs_finetuning.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(\"Saved to ../results/transfer/feature_extraction_vs_finetuning.png\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. Эксперимент 3: Разные Learning Rates\n",
    "### Тестируем разные значения LR для feature extraction"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Результаты для разных LR\n",
    "results_lr = {}\n",
    "\n",
    "learning_rates = [0.0001, 0.001, 0.01]\n",
    "\n",
    "for lr in learning_rates:\n",
    "    print(f\"\\n{'='*60}\")\n",
    "    print(f\"Training with LR = {lr}\")\n",
    "    print(f\"{'='*60}\")\n",
    "    \n",
    "    # Создаем модель\n",
    "    model = get_model(best_model_name, num_classes=NUM_CLASSES, \n",
    "                     pretrained=True, mode='feature_extraction')\n",
    "    \n",
    "    # Оптимизатор\n",
    "    criterion = nn.CrossEntropyLoss()\n",
    "    optimizer = optim.Adam(model.get_trainable_params(), lr=lr)\n",
    "    \n",
    "    # Обучение (меньше эпох для экономии времени)\n",
    "    save_path = f'../checkpoints/transfer_lr_{lr}.pth'\n",
    "    \n",
    "    trained_model, history = train_model(\n",
    "        model, dataloaders, criterion, optimizer,\n",
    "        num_epochs=10, device=device, \n",
    "        patience=3, save_path=save_path\n",
    "    )\n",
    "    \n",
    "    # Сохраняем результаты\n",
    "    results_lr[lr] = {\n",
    "        'history': history,\n",
    "        'best_val_acc': max(history['val_acc'])\n",
    "    }\n",
    "    \n",
    "    print(f\"Best validation accuracy: {results_lr[lr]['best_val_acc']:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Визуализация влияния Learning Rate\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "for lr in learning_rates:\n",
    "    history = results_lr[lr]['history']\n",
    "    \n",
    "    # Loss\n",
    "    axes[0].plot(history['val_loss'], label=f'LR = {lr}', marker='o', markersize=4)\n",
    "    \n",
    "    # Accuracy\n",
    "    axes[1].plot(history['val_acc'], label=f'LR = {lr}', marker='o', markersize=4)\n",
    "\n",
    "axes[0].set_xlabel('Epoch')\n",
    "axes[0].set_ylabel('Validation Loss')\n",
    "axes[0].set_title('Learning Rate Comparison - Loss')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True)\n",
    "\n",
    "axes[1].set_xlabel('Epoch')\n",
    "axes[1].set_ylabel('Validation Accuracy')\n",
    "axes[1].set_title('Learning Rate Comparison - Accuracy')\n",
    "axes[1].legend()\n",
    "axes[1].grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../results/transfer/learning_rates.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(\"Saved to ../results/transfer/learning_rates.png\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. Эксперимент 4: Different LR для разных частей модели\n",
    "### Используем разные learning rates для features и classifier"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Результаты для different LR\n",
    "results_different_lr = {}\n",
    "\n",
    "# Конфигурации: (base_lr для features, classifier_lr для classifier)\n",
    "lr_configs = [\n",
    "    (0.0001, 0.001, 'Standard (0.0001 / 0.001)'),\n",
    "    (0.00001, 0.001, 'Low Base (0.00001 / 0.001)'),\n",
    "    (0.0001, 0.01, 'High Classifier (0.0001 / 0.01)')\n",
    "]\n",
    "\n",
    "for base_lr, classifier_lr, label in lr_configs:\n",
    "    print(f\"\\n{'='*60}\")\n",
    "    print(f\"Training with {label}\")\n",
    "    print(f\"{'='*60}\")\n",
    "    \n",
    "    # Создаем модель в режиме fine-tuning\n",
    "    model = get_model(best_model_name, num_classes=NUM_CLASSES, \n",
    "                     pretrained=True, mode='fine_tuning')\n",
    "    \n",
    "    # Оптимизатор с different LR\n",
    "    criterion = nn.CrossEntropyLoss()\n",
    "    optimizer = get_optimizer_with_different_lr(\n",
    "        model.model,  # передаем внутреннюю модель\n",
    "        base_lr=base_lr,\n",
    "        classifier_lr=classifier_lr,\n",
    "        optimizer_type='adam'\n",
    "    )\n",
    "    \n",
    "    # Обучение\n",
    "    save_path = f'../checkpoints/transfer_diff_lr_{base_lr}_{classifier_lr}.pth'\n",
    "    \n",
    "    trained_model, history = train_model(\n",
    "        model, dataloaders, criterion, optimizer,\n",
    "        num_epochs=10, device=device, \n",
    "        patience=3, save_path=save_path\n",
    "    )\n",
    "    \n",
    "    # Сохраняем результаты\n",
    "    results_different_lr[label] = {\n",
    "        'history': history,\n",
    "        'best_val_acc': max(history['val_acc']),\n",
    "        'base_lr': base_lr,\n",
    "        'classifier_lr': classifier_lr\n",
    "    }\n",
    "    \n",
    "    print(f\"Best validation accuracy: {results_different_lr[label]['best_val_acc']:.4f}\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Визуализация Different LR\n",
    "fig, axes = plt.subplots(1, 2, figsize=(15, 5))\n",
    "\n",
    "for label in results_different_lr.keys():\n",
    "    history = results_different_lr[label]['history']\n",
    "    \n",
    "    # Loss\n",
    "    axes[0].plot(history['val_loss'], label=label, marker='o', markersize=4)\n",
    "    \n",
    "    # Accuracy\n",
    "    axes[1].plot(history['val_acc'], label=label, marker='o', markersize=4)\n",
    "\n",
    "axes[0].set_xlabel('Epoch')\n",
    "axes[0].set_ylabel('Validation Loss')\n",
    "axes[0].set_title('Different LR Comparison - Loss')\n",
    "axes[0].legend()\n",
    "axes[0].grid(True)\n",
    "\n",
    "axes[1].set_xlabel('Epoch')\n",
    "axes[1].set_ylabel('Validation Accuracy')\n",
    "axes[1].set_title('Different LR Comparison - Accuracy')\n",
    "axes[1].legend()\n",
    "axes[1].grid(True)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.savefig('../results/transfer/different_lr_comparison.png', dpi=300, bbox_inches='tight')\n",
    "plt.show()\n",
    "\n",
    "print(\"Saved to ../results/transfer/different_lr_comparison.png\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Финальное сравнение всех результатов"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Создаем итоговую таблицу результатов\n",
    "final_results = []\n",
    "\n",
    "# Результаты по архитектурам\n",
    "for model_name in models_to_test:\n",
    "    final_results.append({\n",
    "        'Experiment': f'{model_name} (Feature Extraction)',\n",
    "        'Best Val Accuracy': f\"{results_architectures[model_name]['best_val_acc']:.4f}\",\n",
    "        'Training Time (min)': f\"{results_architectures[model_name]['training_time']/60:.2f}\"\n",
    "    })\n",
    "\n",
    "# Результаты Feature Extraction vs Fine-Tuning\n",
    "for mode in modes:\n",
    "    final_results.append({\n",
    "        'Experiment': f'{best_model_name} ({mode.replace(\"_\", \" \").title()})',\n",
    "        'Best Val Accuracy': f\"{results_modes[mode]['best_val_acc']:.4f}\",\n",
    "        'Training Time (min)': f\"{results_modes[mode]['training_time']/60:.2f}\"\n",
    "    })\n",
    "\n",
    "# Результаты по LR\n",
    "for lr in learning_rates:\n",
    "    final_results.append({\n",
    "        'Experiment': f'{best_model_name} (LR={lr})',\n",
    "        'Best Val Accuracy': f\"{results_lr[lr]['best_val_acc']:.4f}\",\n",
    "        'Training Time (min)': '-'\n",
    "    })\n",
    "\n",
    "# Результаты Different LR\n",
    "for label in results_different_lr.keys():\n",
    "    final_results.append({\n",
    "        'Experiment': f'Different LR: {label}',\n",
    "        'Best Val Accuracy': f\"{results_different_lr[label]['best_val_acc']:.4f}\",\n",
    "        'Training Time (min)': '-'\n",
    "    })\n",
    "\n",
    "# Создаем DataFrame\n",
    "df_results = pd.DataFrame(final_results)\n",
    "\n",
    "print(\"\\n\" + \"=\"*80)\n",
    "print(\"FINAL TRANSFER LEARNING RESULTS\")\n",
    "print(\"=\"*80)\n",
    "print(df_results.to_string(index=False))\n",
    "print(\"=\"*80)\n",
    "\n",
    "# Сохраняем в файл\n",
    "df_results.to_csv('../results/transfer/final_results.txt', index=False, sep='\\t')\n",
    "print(\"\\nSaved to ../results/transfer/final_results.txt\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Находим лучшую конфигурацию\n",
    "best_config = max(final_results, key=lambda x: float(x['Best Val Accuracy']))\n",
    "\n",
    "print(\"\\n🏆 BEST CONFIGURATION:\")\n",
    "print(f\"   Experiment: {best_config['Experiment']}\")\n",
    "print(f\"   Validation Accuracy: {best_config['Best Val Accuracy']}\")\n",
    "print(f\"   Training Time: {best_config['Training Time (min)']} minutes\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Выводы\n",
    "\n",
    "В этом ноутбуке мы:\n",
    "\n",
    "1. **Сравнили архитектуры**: ResNet18 vs VGG16\n",
    "   - Определили, какая модель лучше работает на нашем датасете\n",
    "\n",
    "2. **Feature Extraction vs Fine-Tuning**:\n",
    "   - Feature Extraction: быстрее обучается, но может иметь ниже точность\n",
    "   - Fine-Tuning: медленнее, но потенциально лучшие результаты\n",
    "\n",
    "3. **Эксперименты с Learning Rate**:\n",
    "   - Протестировали разные значения LR\n",
    "   - Нашли оптимальный LR для нашей задачи\n",
    "\n",
    "4. **Different LR для разных частей модели**:\n",
    "   - Использовали разные LR для features и classifier\n",
    "   - Это позволяет более аккуратно настроить pretrained модель\n",
    "\n",
    "Лучшая модель сохранена в `../checkpoints/transfer_best.pth`"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}